# Code for Lecture 4 Hands on Session

In [ ]:
!grbgetlicense

In [3]:

from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# LP： InvestmentStock
The data is in the file "data1.csv"
The model is LP:
\begin{equation}
\begin{array}{ll}
\text{maximize} &  \sum_{i=1}^{N}r_i(s_i-x_i)\\
\text{subject to} &  \sum_{i=1}^{N}(q_i x_i - 0.3\max(q_i-p_i,0)x_i-0.01x_iq_i) \ge K\\
 & 0\le x_i\le s_i, \forall i = 1,...,N
\end{array}
\end{equation}
Default: K=9000

In [4]:
# read data 
data1 = pd.read_csv('data1.csv')
#print(data1)
#print(type(data1))
header = data1.columns
data = data1.values
print(data)
stock, buy_price, cur_share, cur_price, exp_price = multidict({item[0]: (item[1], item[2], item[3],item[4]) 
                                                              for item in data })#print(stock)   #Index of different stocks
#print(stock)
#print(buy_price)   #one parameter, in a dictionary format
#print(buy_price['S2'])
#print(type(buy_price))
#print({item[0]: (item[1], item[2], item[3], item[4]) for item in data })
print(happy)

[['S1' 1.2 1000 2.1 2.0]
 ['S2' 2.1 1000 3.2 3.7]
 ['S3' 3.2 1000 4.1 5.2]
 ['S4' 4.1 1000 5.1 7.1]
 ['S5' 4.5 1000 6.7 9.1]]


NameError: name 'happy' is not defined

In [5]:
# build model 
K = 9e+3 # set target

m1 = Model("investlp")
sell = m1.addVars(stock, ub = cur_share, lb=0, name = "sell")    # Add variables, upper bound is the current share
#print(sell)
obj = sum( exp_price[i] * (cur_share[i] - sell[i]) for i in stock) 
m1.setObjective( obj, GRB.MAXIMIZE)  # Maximize future portfolio value

m1.addConstr( sum(sell[i]*cur_price[i] - 0.3*sell[i]*( max(cur_price[i]-buy_price[i],0)) - 0.01*sell[i]*cur_price[i] 
                       for i in stock) >= K, "target")  
m1.optimize()


def printSolution1():
    if m1.status == GRB.Status.OPTIMAL:
        print('\nNext year expected portfolio: %g' % m1.objVal)
        print('\nSell:')
        sellx = m1.getAttr('x', sell)   # a dictionary
        for i in stock:
            print('%s %g' % (i, sellx[i]))
    else:
        print('No solution')

#printSolution1() 

print(obj)

Academic license - for non-commercial use only - expires 2021-03-22
Using license file /Users/jameschua/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x18620101
Coefficient statistics:
  Matrix range     [2e+00, 6e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [9e+03, 9e+03]
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7100000e+04   5.625000e+02   0.000000e+00      0s
       1    1.5356791e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.535679090e+04
<gurobi.LinExpr: 27100.0 + -2.0 sell[S1] + -3.7 sell[S2] + -5.2 sell[S3] + -7.1 sell[S4] + -9.1 sell[S5]>


## LP： InvestmentBonds
The data is in the file "data3.csv"

In [6]:
# Investment in bonds

# Initialization
capital = 1000000         # Investment capital
cashRate = 0.02;           # Cash Intereset rate

cashRet = (1.0+cashRate)  # Cash return

# Read from bond data file
data2 = pd.read_csv("data2.csv")
header = data2.columns
#print(data2)

#Determine parameter from data 
data2['End Year'] = data2['Available Year'] + data2['Maturity Period'] # Bond matrity date

print(data2)

T = max(data2['End Year'])-1        # Investment Time period
N = data2['Bond'].count()        # Number of bonds


data = data2.values
print(data)
# Extract data to variables and Calculate bond return
# Bond Index, Start Year, Duration of bond, Bond interest rate per year, Bond limit, End Year, Bond return
bondName, bondStYear, bondDuration, bondRate, bondLimit, bondEndYear, bondRet = multidict(
    {item[0]: [item[1], item[2], item[3], item[4], item[5], (1+item[3])**(item[2])] for item in data })

print(bondRet)


  Bond  Available Year  Maturity Period  Annual Interest Rate     Limit  \
0   B1               1                4                   0.03  1000000   
1   B2               5                1                   0.04   200000   
2   B3               2                4                   0.05   500000   
3   B4               2                3                   0.06   200000   

   End Year  
0         5  
1         6  
2         6  
3         5  
[['B1' 1 4 0.03 1000000 5]
 ['B2' 5 1 0.04 200000 6]
 ['B3' 2 4 0.05 500000 6]
 ['B4' 2 3 0.06 200000 5]]
{'B1': 1.1255088100000001, 'B2': 1.04, 'B3': 1.2155062500000002, 'B4': 1.191016}


In [48]:
# Declare Model
m2 = Model('BondCash Invest')

# Declaring variables
bond = m2.addVars(bondName, ub = bondLimit, name = 'Bond') # Bond and  Bond Limit
cash = m2.addVars(T, name = 'Cash') # Cash
# Defining constraints
# Year 1
m2.addConstr( cash[0] + quicksum(bond[j] for j in bondName if bondStYear[j] ==1) <= capital)

# Year 2 to T
for t in range(1,T):
    m2.addConstr(cash[t] + quicksum(bond[j] for j in bondName if bondStYear[j]==t+1) == 
                  cashRet*cash[t-1] + quicksum(bondRet[j]*bond[j] for j in bondName if bondEndYear[j]==t))

# Setting the objective
m2.setObjective(cashRet*cash[T-1] + quicksum(bondRet[j]*bond[j] for j in bondName if bondEndYear[j]==T+1), GRB.MAXIMIZE)

# Solving the optimization problem
m2.optimize()

# Printing the optimal solutions obtained
def printSolution2():
    if m1.status == GRB.Status.OPTIMAL:
        print('\nOptimal Investment Return : %g' % m2.objVal)
        print('\nBond:')
        bondx = m2.getAttr('x', bond)
        for i in bond:
            print('%s %g' % (i, bondx[i]))
        cashx = m2.getAttr('x', cash)  
        for i in cash:
            print('%s %g' % (i+1, cashx[i]))
    else:
        print('No solution')

printSolution2() 

Optimize a model with 5 rows, 9 columns and 13 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+05, 1e+06]
  RHS range        [1e+06, 1e+06]
Presolve removed 5 rows and 9 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1746178e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal objective  1.174617848e+06

Optimal Investment Return : 1.17462e+06

Bond:
B1 0
B2 200000
B3 500000
B4 0
1 1e+06
2 520000
3 530400
4 541008
5 351828
